# Microtuble Analysis Project - Track Feature Extraction

### Imports

In [ ]:
#IMPORTS
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pims
from skimage import io
from skimage.measure import label, regionprops
from skimage.color import label2rgb
from skimage.util import random_noise
import trackpy as tp
import math
from scipy.spatial import distance
from numpy import linalg as LA
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
import pickle
from scipy.ndimage.morphology import distance_transform_edt
import math
from track_analysis_functions_v1 import *
from PIL import Image
import glob
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### Load and save data

In [ ]:
#List of Folders to Analyze
folders = glob.glob('./example_data/')

In [ ]:
#PHASE 1
for folder in folders:
    # Load all track data
    #root_dir = directory + folder + "/"
    root_dir = folder + "/"
    track_dir = root_dir + "track_matrices"
    mask_dir = root_dir + "masks"
    pickle_dir = root_dir + "pickle_objs/"

    file_names = os.listdir(track_dir)

    try:
        file_names.remove('.DS_Store')
    except:
        pass
        
    label_imgs = []
    mask_imgs = []
    movie_names = []
    t0_tracks = []
    frame_lengths = []
    print("Loading data: " + folder)

    img_id = 0
    for name in file_names:
        new_name = name.split(".csv")[0]
        movie_names.append(name)
        im = Image.open(mask_dir+"/"+new_name+".tif")
        img_mask = np.array(im)
        if img_mask[0,0] > 0:
            img_mask = ~img_mask
        #img_mask = io.imread(mask_dir+"/"+new_name+".tif", as_gray=True)
        mask_imgs.append(img_mask)
        label_imgs.append(label(img_mask, background=0))
        #Loading Track Data as dataframes
        t = pd.read_csv(track_dir+"/"+new_name+".csv", header=None)
        t.columns = ["x", "y", "mass", "frame", "particle"]
        t['img_id'] = img_id
        t0_tracks.append(t)
        frame_lengths.append(np.max(t['frame']))
        img_id+=1

    pickle.dump(file_names, open(pickle_dir + "file_names.p", "wb"))
    
    ##################################################
    print("\tt1_tracks: removing tracks less than a certain length")
    t1_tracks = []
    for i,temp_df in enumerate(t0_tracks):
        run_length = 3
        if frame_lengths[i] >40:
            run_length = 5
        t1 = tp.filter_stubs(temp_df, run_length)
        t1_tracks.append(t1) 

    ##################################################
    print("\tt2_tracks: identify the cell for each track")
    t2_tracks = []

    for i,temp_df in enumerate(t1_tracks):
        #print("Identifying Cells for Track" , i)
        t2 = identifyCellFromTrack(t1_tracks[i], label_imgs[i])
        print(i, len(t2))
        t2_tracks.append(t2) 
    pickle.dump(t2_tracks, open(pickle_dir + "t2_tracks.p", "wb"))
    
    t2_tracks= pickle.load(open(pickle_dir + "t2_tracks.p", "rb"))
    res_framerate_file = root_dir + "external_data/image_resolution_framerate.csv"
    res_framerate = pd.read_csv(res_framerate_file, header=None)

    t2_2_tracks = []
    for i,temp_df in enumerate(t2_tracks):
        print(file_names[i])
        row_val = res_framerate.loc[res_framerate[0] == file_names[i]]
        new_df = pd.concat([temp_df, 
                        pd.DataFrame(columns=['new_x', 'new_y','new_frame', 'delta_t', 'resolution'])])
        res = list(row_val[1])[0]
        framerate = list(row_val[2])[0]
        new_x = np.divide(temp_df['x'], res)
        new_y = np.divide(temp_df['y'], res)
        new_frame = np.multiply(temp_df['frame'], framerate)
        new_df['new_x'] = new_x
        new_df['new_y'] = new_y
        new_df['new_frame'] = new_frame
        new_df['delta_t'] = framerate
        new_df['resolution'] = res
        t2_2_tracks.append(new_df)

    t2_tracks = t2_2_tracks
    pickle.dump(t2_tracks, open(pickle_dir + "t2_tracks.p", "wb"))

    ##################################################
    print("\tt2_features: Computing Motion and mass features from each track")   
    t2_features = []
    for i,temp_df in enumerate(t2_tracks):
        #print("Calculating Basic Features for Track", i)
        t2_feat = generate_features_df(temp_df)
        t2_feat = calc_mass_features(temp_df, t2_feat)
        t2_feat = calc_motion_features(temp_df, t2_feat)
        t2_features.append(t2_feat)
        
    pickle.dump(t2_features, open(pickle_dir + "t2_features.p", "wb"))
    
    #t2_tracks = pickle.load(open(pickle_dir + "t2_tracks.p", "rb"))
    #t2_features = pickle.load(open(pickle_dir + "t2_features.p", "rb"))
    
    ##################################################
    print("\tt3_tracks and t3_features: filtering tracks based on features values")
    t3_tracks = []
    t3_features = []
    for i,temp_df in enumerate(t2_tracks):
        #print("Filtering based on Features for Track", i)
        t3 = temp_df
        t3_feat = t2_features[i]
        t3 = filter_track_from_feat(t3, t3_feat, 'new_displacement', min_val=0.5)
        t3_feat = filter_traj_from_feat(t3_feat, 'new_displacement', min_val=0.5)

        t3 = filter_track_from_feat(t3, t3_feat, 'new_persistance', min_val=0.6)
        t3_feat = filter_traj_from_feat(t3_feat, 'new_persistance', min_val=0.6)
        #t3 = filter_track_from_feat(t3, t3_feat, 'curvature', max_val = 0.5)
        #t3_feat = filter_traj_from_feat(t3_feat, 'curvature', max_val = 0.5)

        t3_tracks.append(t3)
        t3_features.append(t3_feat)

    pickle.dump(t3_tracks, open(pickle_dir + "t3_tracks.p", "wb"))
    pickle.dump(t3_features, open(pickle_dir + "t3_features.p", "wb"))
    
    

In [ ]:
#PHASE 2
for folder in folders:
    ##################################################
    # Load all track data
    #root_dir = directory + folder + "/"
    root_dir = folder + "/"
    track_dir = root_dir + "track_matrices"
    mask_dir = root_dir + "masks"
    pickle_dir = root_dir + "pickle_objs/"

    file_names = pickle.load(open(pickle_dir + "file_names.p", "rb"))
    
    img_sequences = []
    label_imgs = []
    mask_imgs = []
    t0_tracks = []
    
    print("Loading data: " + folder)
    img_id = 0
    for name in file_names:
        new_name = name.split(".csv")[0]
        try:
            #Loading Mask Data
            im = Image.open(mask_dir+"/"+new_name+".tif")
            img_mask = np.array(im)
            if img_mask[0,0] > 0:
                img_mask = ~img_mask
            #img_mask = io.imread(mask_dir+"/"+new_name+".tif", as_gray=True)
            mask_imgs.append(img_mask)
            label_imgs.append(label(img_mask))
            #img_mask = io.imread(mask_dir+"/"+new_name+".tif")
            #mask_imgs.append(img_mask)
            #label_imgs.append(label(img_mask))
            #Loading Track Data as dataframes
            t = pd.read_csv(track_dir+"/"+new_name+".csv", header=None)
            t.columns = ["x", "y", "mass", "frame", "particle"]
            t['img_id'] = img_id
            t0_tracks.append(t)
            img_id+=1
        except:
            print(new_name + " was not found")
    
    ##################################################
    print("\tt3_tracks_updated: Calculating Distance Feature")
    t3_tracks = pickle.load(open(pickle_dir + "t3_tracks.p", "rb"))
    t3_features = pickle.load(open(pickle_dir + "t3_features.p", "rb"))
    
    #Computing edge distance
    distance_transforms = []
    for i,temp in enumerate(t3_tracks):
        num_cells = len(np.unique(label_imgs[i])) - 1
        new_dist_transform = []
        for j in range(num_cells):
            cell_id = j+1
            temp_mask = np.multiply(mask_imgs[i],label_imgs[i] == cell_id)
            dist_transf= distance_transform_edt(temp_mask)
            distance_transf_sum = np.sum(dist_transf)
            num_pixels = np.sum(temp_mask)
            if cell_id == 1:
                new_dist_transform = np.divide(dist_transf, (distance_transf_sum/num_pixels))
            else:
                new_dist_transform = np.add(new_dist_transform, np.divide(dist_transf, (distance_transf_sum/num_pixels)))
        distance_transforms.append(new_dist_transform)
    
    t3_temp_features = []
    for i,temp_df in enumerate(t3_tracks):
        unique_tracks = np.array(temp_df['particle'].unique())
        new_df = pd.concat([t3_features[i],pd.DataFrame(columns=['edge_distance'])])
        for index, track_id in enumerate(unique_tracks):
            track_particles = temp_df.loc[temp_df['particle'] == track_id]
            #x and y coordinates are switched in distance trasnform for some reason
            x_vals = np.mean(track_particles['x'])
            y_vals = np.mean(track_particles['y'])
            result = distance_transforms[i][int(y_vals),int(x_vals)]   
            new_df.loc[track_id]['edge_distance'] = result
        t3_temp_features.append(new_df)
    t3_features = t3_temp_features
    
    #Min axis distance and Maj axis distance features
    majaxis_transform = []
    minaxis_transform = []
    for i,temp in enumerate(t3_tracks):
        num_cells = len(np.unique(label_imgs[i])) - 1
        new_maj_transform = np.zeros((mask_imgs[i].shape[0],mask_imgs[i].shape[1]))
        new_min_transform = np.zeros((mask_imgs[i].shape[0],mask_imgs[i].shape[1]))
        for j in range(num_cells):
            cell_id = j+1
            temp_mask = np.multiply(mask_imgs[i],label_imgs[i] == cell_id)

            props = regionprops(temp_mask)
            props = props[0]
            y0, x0 = props.centroid
            orientation = props.orientation
            x1_maj = x0 + math.cos(orientation) * 0.5 * props.major_axis_length
            y1_maj = y0 - math.sin(orientation) * 0.5 * props.major_axis_length
            x2_maj = x0 - math.cos(orientation) * 0.5 * props.major_axis_length
            y2_maj = y0 + math.sin(orientation) * 0.5 * props.major_axis_length

            x1_min = x0 - math.sin(orientation) * 0.5 * props.minor_axis_length
            y1_min = y0 - math.cos(orientation) * 0.5 * props.minor_axis_length
            x2_min = x0 + math.sin(orientation) * 0.5 * props.minor_axis_length
            y2_min = y0 + math.cos(orientation) * 0.5 * props.minor_axis_length

            new_mask_min = np.zeros((temp_mask.shape[0], temp_mask.shape[1]))
            new_mask_maj = np.zeros((temp_mask.shape[0], temp_mask.shape[1]))

            for x_pos in range(temp_mask.shape[0]):
                for y_pos in range(temp_mask.shape[1]):
                    new_mask_min[x_pos,y_pos] = dist(x1_min,y1_min,x2_min,y2_min,y_pos,x_pos)
                    new_mask_maj[x_pos,y_pos] = dist(x1_maj,y1_maj,x2_maj,y2_maj,y_pos,x_pos)

            new_mask_min = np.multiply(temp_mask,new_mask_min)
            max_value = np.max(new_mask_min)
            new_mask_min = np.divide(new_mask_min, max_value)

            new_mask_maj = np.multiply(temp_mask,new_mask_maj)
            max_value = np.max(new_mask_maj)
            new_mask_maj = np.divide(new_mask_maj, max_value)

            new_maj_transform = np.add(new_mask_maj, new_maj_transform)
            new_min_transform = np.add(new_mask_min, new_min_transform)

        majaxis_transform.append(new_maj_transform)
        minaxis_transform.append(new_min_transform)
        t3_temp_features = []
        
    for i,temp_df in enumerate(t3_tracks):
        unique_tracks = np.array(temp_df['particle'].unique())
        new_df = pd.concat([t3_features[i],pd.DataFrame(columns=['maj_axis_distance', 'min_axis_distance'])])
        for index, track_id in enumerate(unique_tracks):
            track_particles = temp_df.loc[temp_df['particle'] == track_id]
            #x and y coordinates are switched in distance trasnform for some reason
            x_vals = np.mean(track_particles['x'])
            y_vals = np.mean(track_particles['y'])
            result_major = majaxis_transform[i][int(y_vals),int(x_vals)]   
            result_minor = minaxis_transform[i][int(y_vals),int(x_vals)]   
            new_df.loc[track_id]['maj_axis_distance'] = result_major
            new_df.loc[track_id]['min_axis_distance'] = result_minor
        t3_temp_features.append(new_df)
    t3_features = t3_temp_features
    
    pickle.dump(t3_tracks, open(pickle_dir + "t3_tracks.p", "wb"))
    pickle.dump(t3_features, open(pickle_dir + "t3_features.p", "wb"))

    ##################################################
    #t3_tracks = pickle.load(open(pickle_dir + "t3_tracks.p", "rb"))
    #t3_features = pickle.load(open(pickle_dir + "t3_features.p", "rb"))
    
    print("\tcreating track dictionaries")
    track_distance_dicts = {}

    for i,temp_df in enumerate(t3_tracks):
        print(i)
        track_distance_dicts[i] = {}
        unique_cells =  np.array(temp_df['cell_id'].unique())

        for cell in unique_cells:
            #track_distance_dicts[i][cell] = {}
            #print("Creating Dictionary for Cell", cell, "in Image", i)
            temp_cell_df = temp_df.loc[temp_df['cell_id'] == cell]
            unique_tracks = np.array(temp_cell_df['particle'].unique())
            dist_mat = create_distance_matrix(temp_cell_df, unique_tracks)

            for index,track_id in enumerate(unique_tracks):
                #track_distance_dicts[i][cell][track_id] = {}
                track_distance_dicts[i][track_id] = {}

                for srange in [20,200, 5000]:
                    nearby_tracks = [unique_tracks[j] for j, val in enumerate(dist_mat[index,:]) 
                                     if (val<srange and j != index)]
                    #track_distance_dicts[i][cell][track_id][srange] = nearby_tracks
                    track_distance_dicts[i][track_id][srange] = nearby_tracks  
                    
    pickle.dump(track_distance_dicts, open(pickle_dir + "track_distance_dicts.p", "wb"))
    
    #t3_tracks = pickle.load(open(pickle_dir + "t3_tracks.p", "rb"))
    #t3_features = pickle.load(open(pickle_dir + "t3_features.p", "rb"))

    #track_distance_dicts = pickle.load(open(pickle_dir + "track_distance_dicts.p", "rb"))
    
    ##################################################
    # Get Features from Plots Above that Represent Orientation
    print("\tt4_features - Get similarity features")
    t4_features = []
    for i,temp_df in enumerate(t3_tracks):
        print(i)
        #print("Computing Similarity Features for Image",  i)
        unique_tracks = np.array(temp_df['particle'].unique())
        new_df = pd.concat([t3_features[i],pd.DataFrame(columns=['similarity_20', 'similarity_200', 'similarity_5000'])])
        for index, track_id in enumerate(unique_tracks):
            #nearby_tracks = track_distance_dicts[i][track_id][5]
            #if nearby_tracks == []:
            #    continue
            #else:
            #    distances = compute_cosine_distance(temp_df, track_id, nearby_tracks)
            #    average_distance = np.average(np.array(distances)[:,0])
            #    new_df.loc[track_id]['similarity_5'] = average_distance

            #nearby_tracks = track_distance_dicts[i][track_id][10]
            #distances = compute_cosine_distance(temp_df, track_id, nearby_tracks)
            #average_distance = np.average(np.array(distances)[:,0])
            #new_df.loc[track_id]['similarity_10'] = average_distance

            nearby_tracks = track_distance_dicts[i][track_id][20]
            if nearby_tracks == []:
                continue
            distances = compute_cosine_distance(temp_df, track_id, nearby_tracks)
            average_distance = np.average(np.array(distances)[:,0])
            new_df.at[track_id, 'similarity_20'] = -1
            new_df.at[track_id, 'similarity_20'] = average_distance

            #nearby_tracks = track_distance_dicts[i][track_id][100]
            #distances = compute_cosine_distance(temp_df, track_id, nearby_tracks)
            #average_distance = np.average(np.array(distances)[:,0])
            #new_df.loc[track_id]['similarity_100'] = average_distance

            nearby_tracks = track_distance_dicts[i][track_id][200]
            distances = compute_cosine_distance(temp_df, track_id, nearby_tracks)
            average_distance = np.average(np.array(distances)[:,0])
            new_df.at[track_id, 'similarity_200'] = average_distance
            
            nearby_tracks = track_distance_dicts[i][track_id][5000]
            distances = compute_cosine_distance(temp_df, track_id, nearby_tracks)
            average_distance = np.average(np.array(distances)[:,0])
            new_df.at[ track_id, 'similarity_5000'] = average_distance
            
        t4_features.append(new_df)
        
    pickle.dump(t4_features, open(pickle_dir + "t4_features.p", "wb"))
    
    t4_features = pickle.load(open(pickle_dir + "t4_features.p", "rb"))
    
    ##################################################
    print("\tt5_features - Get Orientation Features")
    t5_features = []
    for i,temp_df in enumerate(t3_tracks):
        #print("Computing Major/Minor Axis Features for Image",  i)
        unique_tracks = np.array(temp_df['particle'].unique())
        new_df = pd.concat([t4_features[i],pd.DataFrame(columns=['maj_orient', 'min_orient', 'maj_orient_mag', 'min_orient_mag'])])
        for index, track_id in enumerate(unique_tracks):
            cell_ID = int(temp_df.loc[temp_df['particle']==track_id]['cell_id'].unique()[0])
            distance_maj = getOrientationFromAxis(temp_df, track_id, cell_ID=cell_ID, label_img=label_imgs[i], axis='major')
            distance_min = getOrientationFromAxis(temp_df, track_id, cell_ID=cell_ID, label_img=label_imgs[i], axis='minor')
            
            #For some reason the major and minor orientation got switched for images:
            #M40 0836 comet HT1080 left ear 2020 02 21", "M41 0836 comer Ht1080 rightear 2020 02 21", 
            #"M42 0836 comet Ht1080 no ear 2020 02 25 treated
            '''
            new_df.at[track_id,'maj_orient'] = distance_maj
            new_df.at[track_id,'min_orient'] = distance_min
            new_df.at[track_id,'maj_orient_mag'] = np.absolute(distance_maj)
            new_df.at[track_id,'min_orient_mag'] = np.absolute(distance_min)
            '''
            new_df.at[track_id,'min_orient'] = distance_maj
            new_df.at[track_id,'maj_orient'] = distance_min
            new_df.at[track_id,'min_orient_mag'] = np.absolute(distance_maj)
            new_df.at[track_id,'maj_orient_mag'] = np.absolute(distance_min)


        t5_features.append(new_df)
 
    ##################################################
    print("\tt6_features - Creating Cell IDs")
    t6_features = []
    for i,temp_df in enumerate(t3_tracks):
        #print("Creating Unique Cell ID for image",  i)
        unique_tracks = np.array(temp_df['particle'].unique())
        new_df = pd.concat([t5_features[i],pd.DataFrame(columns=['unique_cell_id', 'unique_id', 'img_id'])])
        for index, track_id in enumerate(unique_tracks):
            cell_ID = int(temp_df.loc[temp_df['particle']==track_id]['cell_id'].unique()[0])
            new_cell_id = float(str(i)+ '.'+str(cell_ID))
            new_df.at[track_id,'unique_cell_id'] = new_cell_id
            new_part_id = float(str(i)+'.'+str(int(track_id)))
            new_df.at[track_id,'unique_id'] = new_part_id
            new_df.at[track_id,'img_id'] = i
        t6_features.append(new_df.dropna())
        
    pickle.dump(t6_features, open(pickle_dir + "t6_features.p", "wb"))
    #t6_features = pickle.load(open(pickle_dir + "t6_features.p", "rb"))
    